# Teste do Modelo Preditivo
Este notebook é apenas um teste com base no seguinte repositório: https://github.com/thmod-on/fiap-challenge-5

### Importação de bibliotecas

In [1]:
import zipfile
import json
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

### Criando métodos

In [2]:
def ler_json(caminho_zip):
    """Metodo criado para ler um arquivo .json que esteja dentro de um .zip
    Estrutura do json deve ser chave -> elemento - > valor 
    ou 
    chave -> categoria -> elemento -> valor

    Args:
        caminho_zip (string): caminho onde está o arquivo .zip

    Returns:
        dataframe: o json sera achatado para retornar um dataframe correspondente, 
        concatenando a categoria.elemento quando necessário
    """
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        nome_arquivo_json = zip_ref.namelist()[0]  # assumindo que só há um arquivo no zip
        with zip_ref.open(nome_arquivo_json) as file:
            dados = json.load(file)

    # Achatar os dados
    linhas = []
    for chave, conteudo in dados.items():
        linha = {"ID": chave}
        # Flatten (achatar) cada subnível com prefixo
        for secao, valores in conteudo.items():
            if isinstance(valores, dict):
                for subchave, subvalor in valores.items():
                    linha[f"{secao}.{subchave}"] = subvalor
            else:
                linha[secao] = valores
        linhas.append(linha)

    # Criar o DataFrame
    df = pd.DataFrame(linhas)
    return df


def ler_json_candidaturas(caminho_zip):
    """Similar ao metodo ler_json, porem, contemplando uma lista de elementos dentro 
    de uma categoria

    Args:
        caminho_zip (string): caminho onde está o arquivo .zip

    Returns:
        dataframe: o json sera achatado para retornar um dataframe correspondente, 
        concatenando a categoria.elemento quando necessário.
        Para o caso de uma lista de elementos, cada um vai gerar uma nova linha para a
        mesma chave inicial
    """
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        nome_arquivo_json = zip_ref.namelist()[0]  # assumindo que só há um arquivo no zip
        with zip_ref.open(nome_arquivo_json) as file:
            dados_candidatura = json.load(file)

    linhas_candidatura = []

    for codigo_vaga, conteudo in dados_candidatura.items():
        prospects = conteudo.get("prospects")

        # Verifica se há prospects e se é uma lista com itens
        if isinstance(prospects, list) and prospects:
            for prospect in prospects:
                linha_candidatura = {
                    'codigo_vaga': codigo_vaga,
                    'titulo_vaga': conteudo.get('titulo', ''),
                    'nome': prospect.get('nome', ''),
                    'codigo_candidato': prospect.get('codigo', ''),
                    'situacao_candidato': prospect.get('situacao_candidado', ''),
                    'data_candidatura': prospect.get('data_candidatura', ''),
                    'ultima_atualizacao': prospect.get('ultima_atualizacao', ''),
                    'comentario': prospect.get('comentario', ''),
                    'recrutador': prospect.get('recrutador', '')
                }
                linhas_candidatura.append(linha_candidatura)

    # Criar o DataFrame final
    df_candidaturas = pd.DataFrame(linhas_candidatura)
    return df_candidaturas

def remover_colunas(df, lst_colunas_manter, str_colunas_apagar):
    """Metodo para remover dinamicamente um conjunto de colunas de um df seguindo um padrao e deixando outras deste mesmo padrao

    Args:
        df (dataframe): dataframe que terá as colunas removidas
        lst_colunas_manter ([string]): lista de strings contendo as colunas que devem ser mantidas
        str_colunas_apagar (string): expressao regular contendo as colunas a serem apagadas

    Returns:
        dataframe: dataframe inicial apos a remocao das colunas solicitadas
    """
    col_apagar = df.columns[df.columns.str.contains(str_colunas_apagar)]
    col_apagar_final = [coluna for coluna in col_apagar if coluna not in lst_colunas_manter]
    df = df.drop(columns=col_apagar_final)
    return df

def exibir_head_df(nome_tabela, df):
    """Metodo para acompanhamento de dataframes

    Args:
        nome_tabela (string): nome a ser exibido no 'cabecalho'
        df (dataframe): dataframe a ser exibido
    """
    tabela = nome_tabela
    print('\n','-'*(len(tabela)+6))
    print('| ', tabela, ' |')
    print('-'*(len(tabela)+6))
    print(df.head(10))

### Carga dos dados

In [3]:
path_applicants = 'applicants.zip'
path_vagas = 'vagas.zip'
path_prospects = 'prospects.zip'

df_applicants = ler_json(path_applicants)
df_vagas      = ler_json(path_vagas)
df_prospects  = ler_json_candidaturas(path_prospects)

### Análise exploratória e pré-processamento dos dados

#### Applicants

In [4]:
df_applicants.shape

(42482, 58)

In [5]:
df_applicants.head()

,ID,infos_basicas.telefone_recado,infos_basicas.telefone,infos_basicas.objetivo_profissional,infos_basicas.data_criacao,infos_basicas.inserido_por,infos_basicas.email,infos_basicas.local,infos_basicas.sabendo_de_nos_por,infos_basicas.data_atualizacao,...,cargo_atual.id_ibrati,cargo_atual.email_corporativo,cargo_atual.cargo_atual,cargo_atual.projeto_atual,cargo_atual.cliente,cargo_atual.unidade,cargo_atual.data_admissao,cargo_atual.data_ultima_promocao,cargo_atual.nome_superior_imediato,cargo_atual.email_superior_imediato
0,31000,,(11) 97048-2708,,10-11-2021 07:29:49,Luna Correia,carolina_aparecida@gmail.com,,,10-11-2021 07:29:49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31001,,(11) 93723-4396,Analista Administrativo,10-11-2021 08:56:16,Laura Pacheco,eduardo_rios@hotmail.com,"São Paulo, São Paulo",Outros,11-11-2021 11:10:31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,31002,,(11) 92399-9824,Administrativo | Financeiro,10-11-2021 09:01:00,Laura Pacheco,pedro_henrique_carvalho@gmail.com,"São Paulo, São Paulo",Anúncio,10-11-2021 11:42:36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31003,,(11) 98100-1727,Área administrativa,10-11-2021 09:08:13,Laura Pacheco,thiago_barbosa@hotmail.com,"São Paulo, São Paulo",Site de Empregos,10-11-2021 16:04:51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,31004,,(11) 92517-2678,,10-11-2021 09:18:46,Maria Clara Pires,diogo_das_neves@hotmail.com,,,10-11-2021 09:22:03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_applicants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42482 entries, 0 to 42481
Data columns (total 58 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   ID                                                42482 non-null  object
 1   infos_basicas.telefone_recado                     42482 non-null  object
 2   infos_basicas.telefone                            42482 non-null  object
 3   infos_basicas.objetivo_profissional               42482 non-null  object
 4   infos_basicas.data_criacao                        42482 non-null  object
 5   infos_basicas.inserido_por                        42482 non-null  object
 6   infos_basicas.email                               42482 non-null  object
 7   infos_basicas.local                               42482 non-null  object
 8   infos_basicas.sabendo_de_nos_por                  42482 non-null  object
 9   infos_basicas.data_atualizac

In [7]:
print(df_applicants.isnull().sum())

ID                                                      0
infos_basicas.telefone_recado                           0
infos_basicas.telefone                                  0
infos_basicas.objetivo_profissional                     0
infos_basicas.data_criacao                              0
infos_basicas.inserido_por                              0
infos_basicas.email                                     0
infos_basicas.local                                     0
infos_basicas.sabendo_de_nos_por                        0
infos_basicas.data_atualizacao                          0
infos_basicas.codigo_profissional                       0
infos_basicas.nome                                      0
informacoes_pessoais.data_aceite                        0
informacoes_pessoais.nome                               0
informacoes_pessoais.cpf                                0
informacoes_pessoais.fonte_indicacao                    0
informacoes_pessoais.email                              0
informacoes_pe

In [8]:
df_applicants.nunique()

ID                                                  42482
infos_basicas.telefone_recado                           1
infos_basicas.telefone                              41026
infos_basicas.objetivo_profissional                  8368
infos_basicas.data_criacao                          41708
infos_basicas.inserido_por                             63
infos_basicas.email                                 41482
infos_basicas.local                                   721
infos_basicas.sabendo_de_nos_por                       14
infos_basicas.data_atualizacao                      41761
infos_basicas.codigo_profissional                   42482
infos_basicas.nome                                  40686
informacoes_pessoais.data_aceite                    11013
informacoes_pessoais.nome                           40686
informacoes_pessoais.cpf                                1
informacoes_pessoais.fonte_indicacao                 2072
informacoes_pessoais.email                          41482
informacoes_pe

In [9]:
# Removendo colunas que consideramos nao influenciar na analise

col_applicants_manter = ['informacoes_pessoais.pcd', 'informacoes_profissionais.titulo_profissional',
                         'informacoes_profissionais.area_atuacao', 'informacoes_profissionais.nivel_profissional',
                         'informacoes_profissionais.remuneracao']

df_applicants = remover_colunas(df_applicants,
                                col_applicants_manter,
                                'infos_basicas\.|informacoes_pessoais\.|informacoes_profissionais\.|cargo_atual\.|formacao_e_idiomas.instituicao_ensino_superior|formacao_e_idiomas.cursos')

df_applicants.drop(columns=['cv_pt', 'cv_en',
                            'formacao_e_idiomas.ano_conclusao', 'formacao_e_idiomas.outro_curso'
                             ], inplace=True)

In [10]:
len(df_applicants.columns)

10

In [11]:
cardinalidade_applicants = df_applicants.nunique()
colunas_baixa_card_applicants = cardinalidade_applicants[cardinalidade_applicants <= 30].index
len(colunas_baixa_card_applicants)

5

In [12]:
df_applicants_2 = df_applicants[['ID'] + list(colunas_baixa_card_applicants)]
df_applicants_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42482 entries, 0 to 42481
Data columns (total 6 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   ID                                            42482 non-null  object
 1   informacoes_pessoais.pcd                      42482 non-null  object
 2   informacoes_profissionais.nivel_profissional  42482 non-null  object
 3   formacao_e_idiomas.nivel_academico            42482 non-null  object
 4   formacao_e_idiomas.nivel_ingles               42482 non-null  object
 5   formacao_e_idiomas.nivel_espanhol             42482 non-null  object
dtypes: object(6)
memory usage: 1.9+ MB


#### Prospects

In [13]:
df_prospects.shape

(53759, 9)

In [14]:
df_prospects.head()

,codigo_vaga,titulo_vaga,nome,codigo_candidato,situacao_candidato,data_candidatura,ultima_atualizacao,comentario,recrutador
0,4530,CONSULTOR CONTROL M,José Vieira,25632,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira
1,4530,CONSULTOR CONTROL M,Srta. Isabela Cavalcante,25529,Encaminhado ao Requisitante,22-03-2021,23-03-2021,"encaminhado para - R$ 6.000,00 – CLT Full , n...",Ana Lívia Moreira
2,4531,2021-2607395-PeopleSoft Application Engine-Dom...,Sra. Yasmin Fernandes,25364,Contratado pela Decision,17-03-2021,12-04-2021,Data de Inicio: 12/04/2021,Juliana Cassiano
3,4531,2021-2607395-PeopleSoft Application Engine-Dom...,Alexia Barbosa,25360,Encaminhado ao Requisitante,17-03-2021,17-03-2021,,Juliana Cassiano
4,4533,2021-2605708-Microfocus Application Life Cycle...,Arthur Almeida,26338,Contratado pela Decision,29-04-2021,18-05-2021,,Stella Vieira


In [15]:
df_prospects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53759 entries, 0 to 53758
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   codigo_vaga         53759 non-null  object
 1   titulo_vaga         53759 non-null  object
 2   nome                53759 non-null  object
 3   codigo_candidato    53759 non-null  object
 4   situacao_candidato  53759 non-null  object
 5   data_candidatura    53759 non-null  object
 6   ultima_atualizacao  53759 non-null  object
 7   comentario          53759 non-null  object
 8   recrutador          53759 non-null  object
dtypes: object(9)
memory usage: 3.7+ MB


In [16]:
print(df_prospects.isnull().sum())

codigo_vaga           0
titulo_vaga           0
nome                  0
codigo_candidato      0
situacao_candidato    0
data_candidatura      0
ultima_atualizacao    0
comentario            0
recrutador            0
dtype: int64


In [17]:
df_prospects.nunique()

codigo_vaga           11279
titulo_vaga            9910
nome                  26247
codigo_candidato      29405
situacao_candidato       21
data_candidatura       1793
ultima_atualizacao     1584
comentario            12194
recrutador               77
dtype: int64

In [18]:
df_prospects.drop(columns=['nome', 'data_candidatura',
                           'recrutador', 'ultima_atualizacao',
                           'comentario', 'recrutador', 'titulo_vaga'
                           ], inplace=True)

In [19]:
df_prospects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53759 entries, 0 to 53758
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   codigo_vaga         53759 non-null  object
 1   codigo_candidato    53759 non-null  object
 2   situacao_candidato  53759 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


#### Vagas

In [20]:
df_vagas.head()

,ID,informacoes_basicas.data_requicisao,informacoes_basicas.limite_esperado_para_contratacao,informacoes_basicas.titulo_vaga,informacoes_basicas.vaga_sap,informacoes_basicas.cliente,informacoes_basicas.solicitante_cliente,informacoes_basicas.empresa_divisao,informacoes_basicas.requisitante,informacoes_basicas.analista_responsavel,...,perfil_vaga.demais_observacoes,perfil_vaga.viagens_requeridas,perfil_vaga.equipamentos_necessarios,beneficios.valor_venda,beneficios.valor_compra_1,beneficios.valor_compra_2,informacoes_basicas.data_inicial,informacoes_basicas.data_final,perfil_vaga.habilidades_comportamentais_necessarias,informacoes_basicas.nome_substituto
0,5185,04-05-2021,00-00-0000,Operation Lead -,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,...,100% Remoto Período – entre 5 – 6 meses,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN
1,5184,04-05-2021,00-00-0000,Consultor PP/QM Sênior,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Yasmin da Rosa,...,• Início: Imediato • Fim: Jan/22,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN
2,5183,04-05-2021,00-00-0000,ANALISTA PL/JR C/ SQL,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Albuquerque,...,Localização: Remoto Perfil: Analista Pleno ou ...,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN
3,5182,04-05-2021,18-05-2021,Technical Architect - 11894809,Não,Nelson-Page,Dr. Raul Monteiro,Decision São Paulo,Cecília Freitas,Clara Rios,...,Budgeted Rate - indicate currency and type (ho...,Não,Notebook padrão -,- p/ mês (168h),fechado,,18-05-2021,17-01-2022,NaN,NaN
4,5181,04-05-2021,00-00-0000,Consultor SAP AUTHORIZATION (BCA) -Pleno / Sênior,Não,Mann and Sons,Cauê Fogaça,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,...,contratação CLT full pela Decision locação rem...,Sim,Nenhum -,-,R$,,NaN,NaN,NaN,NaN


In [21]:
df_vagas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14081 entries, 0 to 14080
Data columns (total 45 columns):
 #   Column                                                Non-Null Count  Dtype 
---  ------                                                --------------  ----- 
 0   ID                                                    14081 non-null  object
 1   informacoes_basicas.data_requicisao                   14081 non-null  object
 2   informacoes_basicas.limite_esperado_para_contratacao  14081 non-null  object
 3   informacoes_basicas.titulo_vaga                       14081 non-null  object
 4   informacoes_basicas.vaga_sap                          14081 non-null  object
 5   informacoes_basicas.cliente                           14081 non-null  object
 6   informacoes_basicas.solicitante_cliente               14081 non-null  object
 7   informacoes_basicas.empresa_divisao                   14081 non-null  object
 8   informacoes_basicas.requisitante                      14081 non-nu

In [22]:
print(df_vagas.isnull().sum())

ID                                                          0
informacoes_basicas.data_requicisao                         0
informacoes_basicas.limite_esperado_para_contratacao        0
informacoes_basicas.titulo_vaga                             0
informacoes_basicas.vaga_sap                                0
informacoes_basicas.cliente                                 0
informacoes_basicas.solicitante_cliente                     0
informacoes_basicas.empresa_divisao                         0
informacoes_basicas.requisitante                            0
informacoes_basicas.analista_responsavel                    0
informacoes_basicas.tipo_contratacao                        0
informacoes_basicas.prazo_contratacao                       0
informacoes_basicas.objetivo_vaga                           0
informacoes_basicas.prioridade_vaga                         0
informacoes_basicas.origem_vaga                             0
informacoes_basicas.superior_imediato                      42
informac

In [23]:
df_vagas.nunique()

ID                                                      14081
informacoes_basicas.data_requicisao                      1543
informacoes_basicas.limite_esperado_para_contratacao     1306
informacoes_basicas.titulo_vaga                         12452
informacoes_basicas.vaga_sap                                2
informacoes_basicas.cliente                               112
informacoes_basicas.solicitante_cliente                   311
informacoes_basicas.empresa_divisao                         2
informacoes_basicas.requisitante                           24
informacoes_basicas.analista_responsavel                   63
informacoes_basicas.tipo_contratacao                       40
informacoes_basicas.prazo_contratacao                       3
informacoes_basicas.objetivo_vaga                           6
informacoes_basicas.prioridade_vaga                         4
informacoes_basicas.origem_vaga                             3
informacoes_basicas.superior_imediato                       1
informac

In [24]:
# Removendo colunas que consideramos nao influenciar na analise

col_vagas_manter = ['informacoes_basicas.titulo_vaga', 'informacoes_basicas.tipo_contratacao',
                    'perfil_vaga.vaga_especifica_para_pcd', 'perfil_vaga.nivel profissional',
                    'perfil_vaga.nivel_academico', 'perfil_vaga.nivel_ingles', 'perfil_vaga.nivel_espanhol',
                    'perfil_vaga.outro_idioma', 'perfil_vaga.areas_atuacao']

df_vagas = remover_colunas(df_vagas,
                           col_vagas_manter,
                           'informacoes_basicas\.|perfil_vaga\.|beneficios\.')

In [25]:
len(df_vagas.columns)

10

In [26]:
cardinalidade_vagas = df_vagas.nunique()
colunas_baixa_card_vagas = cardinalidade_vagas[cardinalidade_vagas <= 30].index
len(colunas_baixa_card_vagas)

6

In [27]:
df_vagas_2 = df_vagas[['ID'] + list(colunas_baixa_card_vagas)]
df_vagas_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14081 entries, 0 to 14080
Data columns (total 7 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   ID                                    14081 non-null  object
 1   perfil_vaga.vaga_especifica_para_pcd  14081 non-null  object
 2   perfil_vaga.nivel profissional        14081 non-null  object
 3   perfil_vaga.nivel_academico           14081 non-null  object
 4   perfil_vaga.nivel_ingles              14081 non-null  object
 5   perfil_vaga.nivel_espanhol            14081 non-null  object
 6   perfil_vaga.outro_idioma              14081 non-null  object
dtypes: object(7)
memory usage: 770.2+ KB


#### Juntando os dataframes
Usando o df_prospects (interação do candidato com uma vaga) como ponte entre df_vagas (informações da vaga) e df_applicants (perfil do candidato)


In [28]:
df_vagas_2.set_index('ID', inplace=True)

df_merged = df_prospects.merge(df_vagas_2, left_on='codigo_vaga', right_index=True)
df_merged.head()

,codigo_vaga,codigo_candidato,situacao_candidato,perfil_vaga.vaga_especifica_para_pcd,perfil_vaga.nivel profissional,perfil_vaga.nivel_academico,perfil_vaga.nivel_ingles,perfil_vaga.nivel_espanhol,perfil_vaga.outro_idioma
0,4530,25632,Encaminhado ao Requisitante,Não,Pleno,Ensino Superior Completo,Nenhum,Nenhum,
1,4530,25529,Encaminhado ao Requisitante,Não,Pleno,Ensino Superior Completo,Nenhum,Nenhum,
2,4531,25364,Contratado pela Decision,Não,Sênior,Ensino Médio Completo,Nenhum,Nenhum,
3,4531,25360,Encaminhado ao Requisitante,Não,Sênior,Ensino Médio Completo,Nenhum,Nenhum,
4,4533,26338,Contratado pela Decision,Não,Sênior,Ensino Médio Completo,Técnico,Fluente,


In [29]:
df_applicants_2.set_index('ID', inplace=True)

df_merged = df_merged.merge(df_applicants_2, left_on='codigo_candidato', right_index=True)
df_merged.head()

,codigo_vaga,codigo_candidato,situacao_candidato,perfil_vaga.vaga_especifica_para_pcd,perfil_vaga.nivel profissional,perfil_vaga.nivel_academico,perfil_vaga.nivel_ingles,perfil_vaga.nivel_espanhol,perfil_vaga.outro_idioma,informacoes_pessoais.pcd,informacoes_profissionais.nivel_profissional,formacao_e_idiomas.nivel_academico,formacao_e_idiomas.nivel_ingles,formacao_e_idiomas.nivel_espanhol
0,4530,25632,Encaminhado ao Requisitante,Não,Pleno,Ensino Superior Completo,Nenhum,Nenhum,,,,,,
1,4530,25529,Encaminhado ao Requisitante,Não,Pleno,Ensino Superior Completo,Nenhum,Nenhum,,Não,,Ensino Superior Completo,Intermediário,Básico
2,4531,25364,Contratado pela Decision,Não,Sênior,Ensino Médio Completo,Nenhum,Nenhum,,Não,,Ensino Superior Completo,Avançado,Intermediário
3,4531,25360,Encaminhado ao Requisitante,Não,Sênior,Ensino Médio Completo,Nenhum,Nenhum,,,,,,
4,4533,26338,Contratado pela Decision,Não,Sênior,Ensino Médio Completo,Técnico,Fluente,,,,,,


#### Tratamento de valores nulos ou vazios

In [30]:
# Colunas a excluir da substituição
colunas_excluidas = ['codigo_vaga', 'codigo_candidato']

# Colunas que serão modificadas
colunas_para_tratar = df_merged.columns.difference(colunas_excluidas)

# Substituindo nulos e strings vazias por 'nenhum' nas colunas selecionadas
df_merged[colunas_para_tratar] = df_merged[colunas_para_tratar].fillna('nenhum').replace('', 'nenhum')

df_merged.head()

,codigo_vaga,codigo_candidato,situacao_candidato,perfil_vaga.vaga_especifica_para_pcd,perfil_vaga.nivel profissional,perfil_vaga.nivel_academico,perfil_vaga.nivel_ingles,perfil_vaga.nivel_espanhol,perfil_vaga.outro_idioma,informacoes_pessoais.pcd,informacoes_profissionais.nivel_profissional,formacao_e_idiomas.nivel_academico,formacao_e_idiomas.nivel_ingles,formacao_e_idiomas.nivel_espanhol
0,4530,25632,Encaminhado ao Requisitante,Não,Pleno,Ensino Superior Completo,Nenhum,Nenhum,nenhum,nenhum,nenhum,nenhum,nenhum,nenhum
1,4530,25529,Encaminhado ao Requisitante,Não,Pleno,Ensino Superior Completo,Nenhum,Nenhum,nenhum,Não,nenhum,Ensino Superior Completo,Intermediário,Básico
2,4531,25364,Contratado pela Decision,Não,Sênior,Ensino Médio Completo,Nenhum,Nenhum,nenhum,Não,nenhum,Ensino Superior Completo,Avançado,Intermediário
3,4531,25360,Encaminhado ao Requisitante,Não,Sênior,Ensino Médio Completo,Nenhum,Nenhum,nenhum,nenhum,nenhum,nenhum,nenhum,nenhum
4,4533,26338,Contratado pela Decision,Não,Sênior,Ensino Médio Completo,Técnico,Fluente,nenhum,nenhum,nenhum,nenhum,nenhum,nenhum


#### Criando a variável alvo

In [31]:
# Tratando os dados a serem utilizados como 'target'

lst_situacoes_aprovado = ['Encaminhado ao Requisitante', 'Contratado pela Decision', 'Contratado como Hunting']
lst_situacoes_reprovado = ['Prospect', 'Não Aprovado pelo Cliente','Não Aprovado pelo RH', 'Não Aprovado pelo Requisitante','Desistiu da Contratação', 'Entrevista com Cliente', 'Em avaliação pelo RH', 'Aprovado', 'Proposta Aceita','Encaminhar Proposta']
lst_situacoes_remover = ['Inscrito', 'Desistiu', 'Entrevista Técnica', 'Sem interesse nesta vaga', 'Documentação PJ','Documentação CLT', 'Documentação Cooperado','Recusado']

In [32]:
# Removendo as linhas com dados irrelevantes na coluna target

df_merged = df_merged[~df_merged['situacao_candidato'].isin(lst_situacoes_remover)]

In [33]:
df_merged['aprovado'] = df_merged['situacao_candidato'].isin(lst_situacoes_aprovado).astype(int)
df_merged.head()

,codigo_vaga,codigo_candidato,situacao_candidato,perfil_vaga.vaga_especifica_para_pcd,perfil_vaga.nivel profissional,perfil_vaga.nivel_academico,perfil_vaga.nivel_ingles,perfil_vaga.nivel_espanhol,perfil_vaga.outro_idioma,informacoes_pessoais.pcd,informacoes_profissionais.nivel_profissional,formacao_e_idiomas.nivel_academico,formacao_e_idiomas.nivel_ingles,formacao_e_idiomas.nivel_espanhol,aprovado
0,4530,25632,Encaminhado ao Requisitante,Não,Pleno,Ensino Superior Completo,Nenhum,Nenhum,nenhum,nenhum,nenhum,nenhum,nenhum,nenhum,1
1,4530,25529,Encaminhado ao Requisitante,Não,Pleno,Ensino Superior Completo,Nenhum,Nenhum,nenhum,Não,nenhum,Ensino Superior Completo,Intermediário,Básico,1
2,4531,25364,Contratado pela Decision,Não,Sênior,Ensino Médio Completo,Nenhum,Nenhum,nenhum,Não,nenhum,Ensino Superior Completo,Avançado,Intermediário,1
3,4531,25360,Encaminhado ao Requisitante,Não,Sênior,Ensino Médio Completo,Nenhum,Nenhum,nenhum,nenhum,nenhum,nenhum,nenhum,nenhum,1
4,4533,26338,Contratado pela Decision,Não,Sênior,Ensino Médio Completo,Técnico,Fluente,nenhum,nenhum,nenhum,nenhum,nenhum,nenhum,1


In [34]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38970 entries, 0 to 53757
Data columns (total 15 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   codigo_vaga                                   38970 non-null  object
 1   codigo_candidato                              38970 non-null  object
 2   situacao_candidato                            38970 non-null  object
 3   perfil_vaga.vaga_especifica_para_pcd          38970 non-null  object
 4   perfil_vaga.nivel profissional                38970 non-null  object
 5   perfil_vaga.nivel_academico                   38970 non-null  object
 6   perfil_vaga.nivel_ingles                      38970 non-null  object
 7   perfil_vaga.nivel_espanhol                    38970 non-null  object
 8   perfil_vaga.outro_idioma                      38970 non-null  object
 9   informacoes_pessoais.pcd                      38970 non-null  object
 10  inf

### Modelo Random Forest

In [35]:
X = df_merged.drop(columns=['aprovado'])
y = df_merged['aprovado']

In [36]:
colunas_categoricas = X.select_dtypes(include='object').columns.tolist()
colunas_numericas = X.select_dtypes(include='number').columns.tolist()

In [37]:
# Separando os dados em treino e teste

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Pipeline

In [38]:
# Pipeline para variáveis categóricas

cat_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='desconhecido')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Pipeline para variáveis numéricas

num_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Combinando os pipelines em um transformador de colunas

preprocessador = ColumnTransformer(transformers=[
    ('cat', cat_pipeline, colunas_categoricas),
    ('num', num_pipeline, colunas_numericas)
])

#### Criando e treinando o modelo

In [39]:
modelo = Pipeline(steps=[
    ('preprocessamento', preprocessador),
    ('classificador', RandomForestClassifier(n_estimators=100, random_state=42))
])

In [40]:
# Treinamento do modelo apenas com os dados de treino

modelo.fit(X_train, y_train)

Pipeline(steps=[('preprocessamento',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='desconhecido',
                                                                                 strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['codigo_vaga',
                                                   'codigo_candidato',
                                                   'situacao_candidato',
                                                   'perfil_vaga.vaga_especifica_para_pcd',
                                                   'perfil_vaga.nivel '...
                                                   'perfil_vaga.outro_idioma',
                                                   'informacoes_pessoais.pcd',
                                                   'informacoes_profissionais.nivel_profissional',
                                                   'formacao_e_idiomas.nivel_academico',
                                                   'formacao_e_idiomas.nivel_ingles',
                                                   'formacao_e_idiomas.nivel_espanhol']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  [])])),
                ('classificador', RandomForestClassifier(random_state=42))])

In [41]:
# Predição nos dados de teste 

y_pred = modelo.predict(X_test)